In [0]:

from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import col, trim, initcap, when, lit, year, month, dayofmonth, concat, concat_ws
import time

In [0]:
catalog_name = "retail_dev"
schema_silver = "silver"
schema_bronze = "bronze"
schema_auditoria ="auditoria"

In [0]:
sql_catalog = f"""
    CREATE CATALOG IF NOT EXISTS {catalog_name}
"""

spark.sql(sql_catalog)

In [0]:
sql_silver = f"""
    CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_silver}
"""

spark.sql(sql_silver)

In [0]:
silver_productos = (
    spark.table(f"{catalog_name}.{schema_bronze}.productos_raw")
    .withColumn("id_producto", F.col("id_producto").cast("bigint"))
    .withColumn("nombre", initcap(F.trim(F.col("nombre"))))
    .withColumn("categoria", initcap(F.trim(F.col("categoria"))))
    .withColumn("precio", F.col("precio").cast("decimal(18,2)"))
    .withColumn("updated_at", F.current_timestamp())
    .dropna(subset=["id_producto"])
    .dropDuplicates(["id_producto"])
)

In [0]:
display(silver_productos)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_silver}.productos (
  id_producto BIGINT,
  nombre STRING,
  categoria STRING,
  precio DECIMAL(18,2),
  updated_at TIMESTAMP
) USING DELTA
""")

In [0]:
silver_productos.createOrReplaceTempView("silver_productos")
merge_start = time.time()
spark.sql(f"""
MERGE INTO {catalog_name}.{schema_silver}.productos tgt
USING silver_productos src
ON tgt.id_producto = src.id_producto
WHEN MATCHED AND src.updated_at > tgt.updated_at THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
""")
merge_end = time.time()

In [0]:
spark.table(f"{catalog_name}.{schema_silver}.productos").show(10)

In [0]:
import time
job_id = int(time.time())           # ID único basado en timestamp
job_run_id = int(time.time() * 100) # más granular
task_run_id = 1                     # o secuencial si prefieres


In [0]:
rows_in = silver_productos.count()
start = time.time()
spark.sql(f"""
INSERT INTO {catalog_name}.{schema_auditoria}.ingestion_log VALUES (
  {job_id},{job_run_id},{task_run_id},
  current_timestamp(), current_timestamp(),
  {int(time.time()-start)},
  'SUCCESS','productos','silver',
  {rows_in},{rows_in},0,0,0,{int(merge_end-merge_start)},current_timestamp()
)
""")

In [0]:
display(spark.table(f"{catalog_name}.{schema_auditoria}.ingestion_log"))
